# 4. Hardware-Software Communication

In this tutorial, you will learn how to send commands to and receive sensor data from a microcontroller using the MQTT protocol.

<img src="../../_static/mqtt-workflow.png" width=50%>

*Workflow diagram of using a MQTT broker with the "publish/subscribe" model to pass temperature data from one client to another.*

## HiveMQ Broker Setup

✅ If you haven't already, follow Step 13b with the Software setup section of the Build Instructions manuscript [🔗 DOI: 10.1016/j.xpro.2023.102329](https://doi.org/10.1016/j.xpro.2023.102329) (pages 11-12 in the PDF) to create your own HiveMQ cluster and security certificate. This is necessary to complete this module's GitHub Classroom assignment.

## Onboard LED

In an earlier module (blink and read), you learned how to blink the onboard LED every two seconds. In this section, you will control the onboard LED using MQTT.

✅ Create a new file on your Pico W microcontroller called `mqtt_led.py` and copy the script below to the file. Ensure that your microcontroller has been set up according to the instructions in the "Running the demo" module earlier in the course. You will also need to upload the [`mqtt_as.py` module](https://github.com/peterhinch/micropython-mqtt/blob/master/mqtt_as/mqtt_as.py) (either in the main directory or in the `lib` folder). Make sure to replace `<your_id_here>` with your course ID (without the brackets). Save the file and click play.

```python
from mqtt_as import MQTTClient, config
from machine import Pin, ADC
import asyncio
from netman import connectWiFi
import ussl
import ntptime
from time import time, gmtime

from secrets import (
    HIVEMQ_HOST,
    HIVEMQ_PASSWORD,
    HIVEMQ_USERNAME,
    PASSWORD,
    SSID,
)

connectWiFi(SSID, PASSWORD, country="US")

# usually would be a device-specific ID, but using course ID for now
COURSE_ID = "<your_id_here>"

# To validate certificates, a valid time is required
ntptime.timeout = 30  # type: ignore
ntptime.host = "pool.ntp.org"
ntptime.settime()

print("Obtaining CA Certificate")
with open("hivemq-com-chain.der", "rb") as f:
    cacert = f.read()
f.close()

# Local configuration
config.update(
    {
        "ssid": SSID,
        "wifi_pw": PASSWORD,
        "server": HIVEMQ_HOST,
        "user": HIVEMQ_USERNAME,
        "password": HIVEMQ_PASSWORD,
        "ssl": True,
        "ssl_params": {
            "server_side": False,
            "key": None,
            "cert": None,
            "cert_reqs": ussl.CERT_REQUIRED,
            "cadata": cacert,
            "server_hostname": HIVEMQ_HOST,
        },
        "keepalive": 30,
    }
)

onboard_led = Pin("LED", Pin.OUT)  # Pico W is slightly different than Pico

send_topic = f"{COURSE_ID}/onboard_led"
receive_topic = f"{COURSE_ID}/onboard_temp"

adcpin = 4
sensor = ADC(adcpin)


def ReadTemperature():
    adc_value = sensor.read_u16()
    volt = (3.3 / 65535) * adc_value
    temperature = 27 - (volt - 0.706) / 0.001721
    return round(temperature, 2)


async def messages(client):  # Respond to incoming messages
    async for topic, msg, retained in client.queue:
        topic = topic.decode()
        msg = msg.decode()
        retained = str(retained)
        print((topic, msg, retained))

        if topic == send_topic:
            if msg == "on":
                onboard_led.on()
            elif msg == "off":
                onboard_led.off()
            elif msg == "toggle":
                onboard_led.toggle()
            temperature = ReadTemperature()
            print(f"Publish {temperature} to {receive_topic}")
            # If WiFi is down the following will pause for the duration.
            await client.publish(receive_topic, f"{temperature}", qos=1)


async def up(client):  # Respond to connectivity being (re)established
    while True:
        await client.up.wait()  # Wait on an Event
        client.up.clear()
        await client.subscribe(send_topic, 1)  # renew subscriptions


async def main(client):
    await client.connect()
    for coroutine in (up, messages):
        asyncio.create_task(coroutine(client))

    start_time = time()
    timestep_s = 5
    while True:
        await asyncio.sleep(timestep_s)
        elapsed_time = round(time() - start_time)
        print(f"Elapsed: {elapsed_time}s")
        # Uncomment the publish statement below and modify it as needed if you
        # want to publish something every `timestep_s` seconds rather than wait
        # for a command as in the messages() function. If WiFi is down the
        # following publish command will pause for the duration.
        
        # await client.publish("some_topic", f"{elapsed_time}", qos=1)


config["queue_len"] = 1  # Use event interface with default queue size
MQTTClient.DEBUG = True  # Optional: print diagnostic messages
client = MQTTClient(config)
del cacert  # to free memory
try:
    asyncio.run(main(client))
finally:
    client.close()  # Prevent LmacRxBlk:1 errors
```

After running the above script, you should see a message something like:

> ```
> MPY: soft reboot
> MAC address: <...>
> connected
> ip = <...>
> Obtaining CA Certificate
> Checking WiFi integrity.
> Got reliable connection
> Connecting to broker.
> Connected to broker.
> Elapsed: 10s
> Elapsed: 20s
> RAM free 119776 alloc 57504
> Elapsed: 30s
> ...
> ```

Then, navigate to the [companion Jupyter notebook](./1.4.1-onboard-led-temp.ipynb) and open it in a Jupyter IDE of your choice. For your convenience, an "Open in Colab" link is provided at the top. If you are running it elsewhere, you will need to manually install the `paho-mqtt` and `matplotlib` packages (i.e., `pip install paho-mqtt matplotlib`).

## How MQTT Works

✅ Read [How MQTT Works](http://www.steves-internet-guide.com/mqtt-works/)

✅ Read about the [Paho MQTT Python client](http://www.steves-internet-guide.com/receiving-messages-mqtt-python-client/). Pay attention to the section about using a `Queue`

✅ Briefly review the [micropython-mqtt repository](https://github.com/peterhinch/micropython-mqtt) and its [Asynchronous MQTT docs](https://github.com/peterhinch/micropython-mqtt/blob/master/mqtt_as/README.md)

✅ Briefly review the [first-in first-out (FIFO) Queue](https://pymotw.com/2/Queue/)

## Additional Resources
- [How to Send and Receive Data Using Raspberry Pi Pico W and MQTT](https://www.tomshardware.com/how-to/send-and-receive-data-raspberry-pi-pico-w-mqtt)
- Alternative to `mqtt_as`: [`micropython-umqtt.simple2` repository](https://github.com/fizista/micropython-umqtt.simple2)
<!-- - [Getting started with your Raspberry Pi Pico W](https://projects.raspberrypi.org/en/projects/get-started-pico-w) -->